In [1]:
import numpy as np
import re
from random import randint

#Importing Data
wordsList = np.load('Numpy Arrays/wordsList.npy')
wordsList = wordsList.tolist()
wordsList = [word.decode('UTF-8') for word in wordsList]
wordVectors = np.load('Numpy Arrays/wordVectors.npy')
#text = np.load('Numpy Arrays/text.npy')
the_labels = np.load('Numpy Arrays/labels.npy')
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

#Helper Functions
def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())
ids = np.load('Numpy Arrays/idsMatrix.npy')
def getTrainBatch():
    label = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1, 1000000)
        arr[i] = ids[num]
        label.append(the_labels[num])
    return arr, label
def getTestBatch():
    label = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1000001, the_labels.shape[0]-1)
        arr[i] = ids[num]
        label.append(the_labels[num])
    return arr, label

In [2]:
batchSize = 512
numClasses = 2
iterations = 10000
learning_rate = 0.0001
maxSeqLength = 20

In [3]:
import tensorflow as tf
with tf.name_scope("Input_Data") as scope:
    labels = tf.placeholder(tf.float32, [batchSize, numClasses], name='Sentiment_Labels')
    input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength], name='Text')

In [4]:
with tf.name_scope("Data_Preprocessing") as scope:
    data = tf.Variable(tf.zeros([batchSize, maxSeqLength, 3]),dtype=tf.float32, name='Embed_Text')
    data = tf.nn.embedding_lookup(wordVectors,input_data)

In [5]:
data = tf.reshape(data, [batchSize, maxSeqLength, 50, 1])

In [6]:
with tf.name_scope("Convolution1") as scope:
    kernel = tf.get_variable(name="weights",shape=[5, 5, 1, 10], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())
    conv = tf.nn.conv2d(data, kernel, padding="SAME", strides=[1,1,1,1])
    sig = tf.nn.relu(conv)
    sig = tf.nn.dropout(sig, keep_prob=0.9)
    max_pool = tf.nn.max_pool(sig, strides=[1,2,2,1], padding="VALID", ksize=[1,3,3,1])

In [7]:
with tf.name_scope("Convolution2") as scope:
    kernel = tf.get_variable(name="weights_2",shape=[5, 5, 10, 10], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())
    conv = tf.nn.conv2d(max_pool, kernel, padding="SAME", strides=[1,1,1,1])
    sig = tf.nn.relu(conv)
    sig = tf.nn.dropout(sig, keep_prob=0.9)

    max_pool = tf.nn.max_pool(sig, strides=[1,2,2,1], padding="VALID", ksize=[1,3,3,1])

In [8]:
max_pool = tf.reshape(max_pool, [batchSize, 4*11*10])

In [9]:
with tf.name_scope("Prediction_Parameters") as scope:
    weight = tf.Variable(tf.truncated_normal([440, numClasses]), name='Weights')
    bias = tf.Variable(tf.constant(0.1, shape=[numClasses]), name='bias')
with tf.name_scope("Prediction") as scope:
    prediction = tf.nn.softmax(tf.matmul(max_pool, weight) + bias)

In [10]:
with tf.name_scope("Accuracy") as scope:
    correctPred = tf.equal(tf.argmax(tf.argmax(prediction,1), axis=-1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [11]:
with tf.name_scope("Loss") as scope:
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=labels))
with tf.name_scope("Train") as scope:
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

import datetime
tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
validation = tf.summary.scalar('Validation', accuracy)
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [12]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())


In [13]:
for i in range(iterations+1):
        #Next Batch of reviews
    nextBatch, nextBatchLabels = getTrainBatch();
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
    #Write summary to Tensorboard
#    if (i % 5 == 0):
#        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
#        writer.add_summary(summary, i)
#        nextTestBatch, nextTestBatchLabels = getTestBatch();
#        val = sess.run(validation, {input_data: nextTestBatch, labels: nextTestBatchLabels})
#        writer.add_summary(val, i)
    if (i%500 == 0):
        print("Loss: " + str(sess.run(loss, {input_data: nextBatch, labels: nextBatchLabels})))
save_path = saver.save(sess, "CNN2/convolution.ckpt", global_step=i)
writer.close()

Loss: 0.82326114
Loss: 0.69630337
Loss: 0.65460193
Loss: 0.6741843
Loss: 0.6424663
Loss: 0.6397988
Loss: 0.63021135
Loss: 0.63932073
Loss: 0.6348468
Loss: 0.62200725
Loss: 0.64260054
Loss: 0.64552003
Loss: 0.6430118
Loss: 0.6490983
Loss: 0.6404112
Loss: 0.6228039
Loss: 0.60368955
Loss: 0.6169435
Loss: 0.61032
Loss: 0.61017287
Loss: 0.6096728


NameError: name 'writer' is not defined

In [14]:
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix
def test(sent):
    x = ['negetive', 'positive']
    sent = getSentenceMatrix(sent)
    pred = sess.run(prediction, {input_data:sent})[0]
    return(pred)
def results():
    arr = np.array([[1, 0]], dtype=np.float32)
    arr_pred = np.array([test("my life sucks")])
    for i in range(512, ids.shape[0], 512):
        arr = np.vstack((arr, the_labels[i-512:i]))
        arr_pred = np.vstack((arr_pred, sess.run(prediction, {input_data:ids[i-512:i]})))
    x = np.argmax(arr_pred, axis=1)
    y = np.argmax(arr, axis=1)
    return((np.sum(np.equal(x, y))/x.shape))

In [15]:
results()

array([0.66145834])